In [59]:
#!/usr/bin/python3.3
# -*-coding:Utf-8 -*


import pandas as pd 
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt 
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'
# reading data
train = pd.read_csv("sources/train.csv")
test = pd.read_csv("sources/test.csv")

In [60]:
#add a new column 'date' convert into datetime for both training and test set
train['date']=0
for i in range(len(train)) :
	train['date'][i]=datetime.strptime(train['datetime'][i],'%Y-%m-%d %H:%M:%S')

train=train.set_index("datetime")


test['date']=0
for i in range(len(test)) :
	test['date'][i]=datetime.strptime(test['datetime'][i],'%Y-%m-%d %H:%M:%S')

test=test.set_index("datetime")

In [61]:
#categorize training set
train_factor = train
train_factor['weather'] = train_factor['weather'].astype('category')
train_factor['holiday'] = train_factor['holiday'].astype('category')
train_factor['workingday'] = train_factor['workingday'].astype('category')
train_factor['season'] = train_factor['season'].astype('category')

#factorize test set
test_factor = test
test_factor['weather'] = test_factor['weather'].astype('category')
test_factor['holiday'] = test_factor['holiday'].astype('category')
test_factor['workingday'] = test_factor['workingday'].astype('category')
test_factor['season'] = test_factor['season'].astype('category')

In [62]:
#create day of week column and hour column and a year colum then convert into category for both training and test set
train_factor['day']=0
train_factor['hour']=0
train_factor['year']=0
for i in range(len(train_factor)) :
    train_factor['day'][i] = train_factor['date'][i].weekday()
    train_factor['hour'][i] = train_factor['date'][i].hour
    train_factor['year'][i]=train_factor['date'][i].year

train_factor['day']=train_factor['day'].astype('category')
train_factor['hour']=train_factor['hour'].astype('category')
train_factor['year']=train_factor['year'].astype('category')

test_factor['day']=0
test_factor['hour']=0
test_factor['year']=0
for i in range(len(test_factor)) :
    test_factor['day'][i] = test_factor['date'][i].weekday()
    test_factor['hour'][i] = test_factor['date'][i].hour
    test_factor['year'][i]=test_factor['date'][i].year

test_factor['day']=test_factor['day'].astype('category')
test_factor['hour']=test_factor['hour'].astype('category')
test_factor['year']=test_factor['year'].astype('category')

In [63]:
#is a day more significant ?
byday=train_factor.groupby('day')
print(byday.describe())
print('Sunday seems to be an important predictor')

                 atemp       casual        count     humidity   registered  \
day                                                                          
0   count  1551.000000  1551.000000  1551.000000  1551.000000  1551.000000   
    mean     24.178507    29.843972   190.390716    62.508059   160.546744   
    std       8.413965    35.909764   180.943376    18.314346   160.634673   
    min       5.305000     0.000000     1.000000    15.000000     0.000000   
    25%      17.425000     4.000000    39.000000    48.000000    33.000000   
    50%      25.000000    16.000000   146.000000    63.000000   120.000000   
    75%      31.060000    44.000000   277.500000    77.000000   221.500000   
    max      42.425000   242.000000   968.000000   100.000000   857.000000   
1   count  1539.000000  1539.000000  1539.000000  1539.000000  1539.000000   
    mean     24.252898    22.979207   189.723847    63.662768   166.744639   
    std       8.500099    25.735518   186.719673    18.890279   

In [64]:
#create Sunday variable and convert to category 
train_factor['Sunday'] = 0 ;
train_factor.ix[train_factor['day']==6,'Sunday']=1

train_factor['Sunday']=train_factor['Sunday'].astype('category')

test_factor['Sunday'] = 0 ;
test_factor.ix[test_factor['day']==6,'Sunday']=1

test_factor['Sunday']=test_factor['Sunday'].astype('category')

In [65]:
train_factor['temp'].describe()

count    10886.00000
mean        20.23086
std          7.79159
min          0.82000
25%         13.94000
50%         20.50000
75%         26.24000
max         41.00000
Name: temp, dtype: float64

In [66]:
#create daypart variable and convert to category (8 category) for both training and test set
train_factor['daypart'] = 8
for i in range(len(train_factor)) :
    currentHour = train_factor['hour'][i]
    if (currentHour==20 and currentHour==21 ):
        train_factor['daypart'][i] = 7
    elif (currentHour==18 and currentHour==19):
        train_factor['daypart'][i] = 6
    elif (currentHour>9 and currentHour<18):
        train_factor['daypart'][i] = 5
    elif (currentHour==9):
        train_factor['daypart'][i] = 4
    elif (currentHour==8 ):
        train_factor['daypart'][i] = 3
    elif (currentHour<8):
        train_factor['daypart'][i] = 2
    elif (currentHour>=22 ):
        train_factor['daypart'][i] = 1
          
train_factor['daypart']=train_factor['daypart'].astype('category')

test_factor['daypart'] = 8
for i in range(len(test_factor)) :
    currentHour = test_factor['hour'][i]
    if (currentHour==20 and currentHour==21 ):
        test_factor['daypart'][i] = 7
    elif (currentHour==18 and currentHour==19):
        test_factor['daypart'][i] = 6
    elif (currentHour>9 and currentHour<18) :
        test_factor['daypart'][i] = 5
    elif (currentHour==9):
        test_factor['daypart'][i] = 4
    elif (currentHour==8) :
        test_factor['daypart'][i] = 3
    elif (currentHour<8):
        test_factor['daypart'][i] = 2
    elif (currentHour>=22) :
        test_factor['daypart'][i] = 1
test_factor['daypart']=test_factor['daypart'].astype('category')

In [67]:
#create ahot () variable and convert to category (4 category) for both training and test set
train_factor['ahot'] = 4
for i in range(len(train_factor)) :
	currentTemp = train_factor['atemp'][i]
	if (currentTemp>=24.24 and currentTemp<31.06 ):
		train_factor['ahot'][i] = 3
	elif (currentTemp>=16.6 and currentTemp<24.24):
		train_factor['ahot'][i] = 2
	elif ( currentTemp<16.6) :
		train_factor['ahot'][i] = 1

train_factor['ahot']=train_factor['ahot'].astype('category')

test_factor['ahot'] = 4
for i in range(len(test_factor)) :
	currentTemp = test_factor['atemp'][i]
	if (currentTemp>=24.24 and currentTemp<31.06 ):
		test_factor['ahot'][i] = 3
	elif (currentTemp>=16.6 and currentTemp<24.24):
		test_factor['ahot'][i] = 2
	elif (currentTemp<16.6) :
		test_factor['ahot'][i] = 1

test_factor['ahot']=test_factor['ahot'].astype('category')

In [68]:
train_factor['temp'].describe()

count    10886.00000
mean        20.23086
std          7.79159
min          0.82000
25%         13.94000
50%         20.50000
75%         26.24000
max         41.00000
Name: temp, dtype: float64

In [69]:
#create ahot () variable and convert to category (4 category) for both training and test set
train_factor['hot'] = 4
for i in range(len(train_factor)) :
	currentTemp = train_factor['temp'][i]
	if (currentTemp>=20.5 and currentTemp<26.24 ):
		train_factor['hot'][i] = 3
	elif (currentTemp>=13.94 and currentTemp<20.5):
		train_factor['hot'][i] = 2
	elif ( currentTemp<13.94) :
		train_factor['hot'][i] = 1

train_factor['hot']=train_factor['hot'].astype('category')

test_factor['hot'] = 4
for i in range(len(test_factor)) :
	currentTemp = test_factor['temp'][i]
	if (currentTemp>=20.5 and currentTemp<26.24 ):
		test_factor['hot'][i] = 3
	elif (currentTemp>=13.94 and currentTemp<20.5):
		test_factor['hot'][i] = 2
	elif (currentTemp<13.94) :
		test_factor['hot'][i] = 1

test_factor['hot']=test_factor['hot'].astype('category')

In [70]:
train_factor.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,date,day,hour,year,Sunday,daypart,ahot,hot
datetime,,,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011-01-01 00:00:00,5,0,2011,0,2,1,1
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011-01-01 01:00:00,5,1,2011,0,2,1,1
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011-01-01 02:00:00,5,2,2011,0,2,1,1
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011-01-01 03:00:00,5,3,2011,0,2,1,1
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011-01-01 04:00:00,5,4,2011,0,2,1,1


In [71]:
#split into weekday and weekend
train_factor_weekday=train_factor[(train_factor['day']!=5)]
train_factor_weekday=train_factor_weekday[(train_factor_weekday['day']!=6)]

test_factor_weekday=test_factor[(test_factor['day']!=5)]
test_factor_weekday=test_factor_weekday[(test_factor_weekday['day']!=6)]

train_factor_weekend=train_factor[(train_factor['day']!=0)]
train_factor_weekend=train_factor_weekend[(train_factor_weekend['day']!=1)]
train_factor_weekend=train_factor_weekend[(train_factor_weekend['day']!=2)]
train_factor_weekend=train_factor_weekend[(train_factor_weekend['day']!=3)]
train_factor_weekend=train_factor_weekend[(train_factor_weekend['day']!=4)]

test_factor_weekend=test_factor[(test_factor['day']!=0)]
test_factor_weekend=test_factor_weekend[(test_factor_weekend['day']!=1)]
test_factor_weekend=test_factor_weekend[(test_factor_weekend['day']!=2)]
test_factor_weekend=test_factor_weekend[(test_factor_weekend['day']!=3)]
test_factor_weekend=test_factor_weekend[(test_factor_weekend['day']!=4)]

In [72]:
#split into casual and registered


In [73]:
#linear regression weekday/weekend 
formula = "count ~ season + workingday + holiday + humidity+ weather + temp + atemp + windspeed + day + hour + year + ahot + hot"

regression_weekday=smf.glm(formula,data=train_factor_weekday,family=sm.families.Poisson(link=sm.families.links.log))
regression_weekday=regression_weekday.fit()
print(regression_weekday.summary2())

test_factor_weekday=test_factor[(test_factor['day']!=5)]
test_factor_weekday=test_factor_weekday[(test_factor_weekday['day']!=6)]

formula = "count ~ season + humidity + weather + temp + atemp + windspeed + day + hour + year + ahot + hot + Sunday"

regression_weekend=smf.glm(formula,data=train_factor_weekend,family=sm.families.Poisson(link=sm.families.links.log))
regression_weekend=regression_weekend.fit()
print(regression_weekend.summary2())


                Results: Generalized linear model
Model:               GLM              AIC:            144956.9543
Link Function:       log              BIC:            26863.2539 
Dependent Variable:  count            Log-Likelihood: -72432.    
Date:                2016-05-24 08:10 LL-Null:        -6.7370e+05
No. Observations:    7723             Deviance:       95587.     
Df Model:            45               Pearson chi2:   9.72e+04   
Df Residuals:        7677             Scale:          1.0000     
Method:              IRLS                                        
-----------------------------------------------------------------
                 Coef.  Std.Err.     z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------
Intercept        1.8349   0.0076  241.9668 0.0000  1.8200  1.8497
season[T.2]      0.2994   0.0033   91.2880 0.0000  0.2930  0.3058
season[T.3]      0.3037   0.0040   76.3823 0.0000  0.2959  0.3115
season[T.4]      0.4462   

In [74]:
train_factor.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,date,day,hour,year,Sunday,daypart,ahot,hot
datetime,,,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011-01-01 00:00:00,5,0,2011,0,2,1,1
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011-01-01 01:00:00,5,1,2011,0,2,1,1
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011-01-01 02:00:00,5,2,2011,0,2,1,1
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011-01-01 03:00:00,5,3,2011,0,2,1,1
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011-01-01 04:00:00,5,4,2011,0,2,1,1


In [75]:
#linear regression all week
formula = "count ~ season + workingday + holiday + humidity+ weather + temp + atemp + windspeed + day + hour + daypart + ahot + hot"

regression_allWeek = smf.glm(formula,data=train_factor,family=sm.families.Poisson(link=sm.families.links.log))
regression_allWeek=regression_allWeek.fit()
print(regression_allWeek.summary2())



                Results: Generalized linear model
Model:               GLM              AIC:            523725.6402
Link Function:       log              BIC:            353233.6917
Dependent Variable:  count            Log-Likelihood: -2.6182e+05
Date:                2016-05-24 08:10 LL-Null:        -9.3511e+05
No. Observations:    10886            Deviance:       4.5398e+05 
Df Model:            46               Pearson chi2:   4.62e+05   
Df Residuals:        10839            Scale:          1.0000     
Method:              IRLS                                        
-----------------------------------------------------------------
                 Coef.  Std.Err.     z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------
Intercept        3.1263   0.0040  783.1872 0.0000  3.1185  3.1341
season[T.2]      0.2585   0.0028   91.9271 0.0000  0.2530  0.2640
season[T.3]      0.2342   0.0034   68.9985 0.0000  0.2276  0.2409
season[T.4]      0.4230   

In [76]:
#prediction weekday/weekend
prediction_weekday=regression_weekday.predict(test_factor_weekday)
prediction_weekday[prediction_weekday<0]=0 # bike demand is positive

prediction_weekend=regression_weekend.predict(test_factor_weekend)
prediction_weekend[prediction_weekend<0]=0 # bike demand is positive



In [77]:
#prediction all week
prediction_allWeek=regression_allWeek.predict(test_factor)
prediction_allWeek[prediction_allWeek<0]=0 # bike demand is positive


In [78]:
#convert to dataframe
prediction_weekend=pd.DataFrame(prediction_weekend)
prediction_weekend.columns=['count']
prediction_weekend["datetime"]=test_factor_weekend.index.values
prediction_weekend = prediction_weekend.set_index("datetime")

prediction_weekday=pd.DataFrame(prediction_weekday)
prediction_weekday.columns=['count']
prediction_weekday["datetime"]=test_factor_weekday.index.values
prediction_weekday = prediction_weekday.set_index("datetime")

prediction_allWeek=pd.DataFrame(prediction_allWeek)
prediction_allWeek.columns=['count']
prediction_allWeek["datetime"]=test_factor.index.values
prediction_allWeek=prediction_allWeek.set_index("datetime")

In [80]:
#write prediction for weekend/weekday
prediction=prediction_weekend;
prediction=prediction.append(prediction_weekday)
prediction.sort_index()
print(prediction)

                          count
datetime                       
2011-01-22 00:00:00   43.743774
2011-01-22 01:00:00   30.162786
2011-01-22 02:00:00   24.087967
2011-01-22 03:00:00   11.520822
2011-01-22 04:00:00    3.789963
2011-01-22 06:00:00    8.331552
2011-01-22 07:00:00   19.837778
2011-01-22 08:00:00   45.706755
2011-01-22 09:00:00   78.379995
2011-01-22 10:00:00  110.799689
2011-01-22 11:00:00  133.965260
2011-01-22 12:00:00  159.469680
2011-01-22 13:00:00  185.274793
2011-01-22 14:00:00  168.155299
2011-01-22 15:00:00  170.993014
2011-01-22 16:00:00  155.975094
2011-01-22 17:00:00  161.485568
2011-01-22 18:00:00  122.804978
2011-01-22 19:00:00  106.039471
2011-01-22 20:00:00   79.181209
2011-01-22 21:00:00   67.899797
2011-01-22 22:00:00   60.811165
2011-01-22 23:00:00   43.876525
2011-01-23 00:00:00   41.588367
2011-01-23 01:00:00   32.861361
2011-01-23 02:00:00   24.569537
2011-01-23 03:00:00   11.828497
2011-01-23 05:00:00    3.688669
2011-01-23 06:00:00    7.672470
2011-01-

In [51]:
import matplotlib.pyplot as plt
prediction['day']=test_factor['day']
prediction['hour']=test_factor['hour']

In [57]:
plt.scatter(prediction['day'],prediction["count"].as_matrix())
plt.xlabel('day')
plt.ylabel('count prediction')
plt.title('Count predict on testing set')
plt.axis([-1,8,-100,1100])
plt.xticks([0,1,2,3,4,5,6],['M','T','W','T','F','S','S'])
plt.grid(True)
plt.show()

In [52]:
plt.scatter(prediction['hour'],prediction["count"].as_matrix())
plt.xlabel('hour')
plt.ylabel('count prediction')
plt.title('Count predict on testing set')
plt.axis([-1,25,-100,1100])
plt.grid(True)
plt.show()

In [27]:
len(train_factor['count'])

10886

In [58]:
plt.scatter(train_factor["day"],train_factor["count"].as_matrix())
plt.xlabel('day')
plt.ylabel('count observed')
plt.title('Count observed on training set')
plt.axis([-1,8,-100,1100])
plt.xticks([0,1,2,3,4,5,6],['M','T','W','T','F','S','S'])
plt.grid(True)
plt.show()

In [56]:
plt.scatter(train_factor["hour"],train_factor["count"].as_matrix())
plt.xlabel('hour')
plt.ylabel('count observed')
plt.title('Count observed on training set')
plt.axis([-1,25,-100,1100])
plt.grid(True)
plt.show()

In [81]:
#write the submission
prediction.to_csv("Résultats/regressionPoissonFactorWeekend_Weekday.csv")


In [64]:
#write prediction for all week
prediction=prediction_allWeek;
prediction.sort_index()
print(prediction)

                          count
datetime                       
2011-01-20 00:00:00   28.639020
2011-01-20 01:00:00   20.639285
2011-01-20 02:00:00   13.982302
2011-01-20 03:00:00    6.862897
2011-01-20 04:00:00    3.841548
2011-01-20 05:00:00   11.394990
2011-01-20 06:00:00   44.730008
2011-01-20 07:00:00  125.131590
2011-01-20 08:00:00  203.117647
2011-01-20 09:00:00  117.002377
2011-01-20 10:00:00   89.872341
2011-01-20 11:00:00  108.316387
2011-01-20 12:00:00  167.124999
2011-01-20 13:00:00  128.976559
2011-01-20 14:00:00  120.178148
2011-01-20 15:00:00  124.874892
2011-01-20 16:00:00  153.980631
2011-01-20 17:00:00  243.271332
2011-01-20 18:00:00  206.176502
2011-01-20 19:00:00  148.622835
2011-01-20 20:00:00  108.330127
2011-01-20 21:00:00   84.041859
2011-01-20 22:00:00   63.140785
2011-01-20 23:00:00   43.529752
2011-01-21 00:00:00   27.431904
2011-01-21 01:00:00   17.330308
2011-01-21 02:00:00    7.937313
2011-01-21 03:00:00    3.825235
2011-01-21 04:00:00    3.498389
2011-01-

In [65]:
#write the submission
prediction.to_csv("Résultats/regressionPoissonFactorAll_Week.csv")


In [23]:
from sklearn.linear_model import LogisticRegression
import numpy as np

In [24]:
lr=LogisticRegression()

In [25]:
#algorithme de sélection de var.
from sklearn.feature_selection import RFE
selecteur=RFE(estimator=lr)

In [ ]:
train_factor.head()

In [ ]:
data_X_weekday=train_factor_weekday.drop(["casual","registered","count","date"],1)
data_Y_weekday=train_factor_weekday['count']
data_Y_weekday=data_Y_weekday.as_matrix()
data_X_weekday=data_X_weekday.as_matrix()

data_X_weekend=train_factor_weekend.drop(["casual","registered","count","date"],1)
data_Y_weekend=train_factor_weekend['count']
data_Y_weekend=data_Y_weekend.as_matrix()
data_X_weekend=data_X_weekend.as_matrix()

In [37]:
predictor_weekday=selecteur.fit(data_X_weekday,data_Y_weekday)
predictor_weekend=selecteur.fit(data_X_weekend,data_Y_weekend)

In [38]:
print(predictor_weekday)
print(predictor_weekend)

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
  estimator_params=None, n_features_to_select=None, step=1, verbose=0)
RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
  estimator_params=None, n_features_to_select=None, step=1, verbose=0)


In [39]:
print(train_factor_weekday.drop(["casual","registered","count","date"],1).head())
print(predictor_weekday.support_)
print(predictor_weekday.ranking_)
print(predictor_weekend.support_)
print(predictor_weekend.ranking_)

                    season holiday workingday weather  temp  atemp  humidity  \
datetime                                                                       
2011-01-03 00:00:00      1       0          1       1  9.02  9.850        44   
2011-01-03 01:00:00      1       0          1       1  8.20  8.335        44   
2011-01-03 04:00:00      1       0          1       1  6.56  6.820        47   
2011-01-03 05:00:00      1       0          1       1  6.56  6.820        47   
2011-01-03 06:00:00      1       0          1       1  5.74  5.305        50   

                     windspeed day hour Sunday daypart ahot hot  
datetime                                                         
2011-01-03 00:00:00    23.9994   0    0      0       0    1   1  
2011-01-03 01:00:00    27.9993   0    1      0       0    1   1  
2011-01-03 04:00:00    26.0027   0    4      0       1    1   1  
2011-01-03 05:00:00    19.0012   0    5      0       1    1   1  
2011-01-03 06:00:00    26.0027   0    6    